# **Final Project**

For our final project we try to forecast stock price on a given day from a sequence from previous days data.

We will try several models, starting from a simple LSTM adapted from our Lab 2 work.  With some modifications:

* Removing the last linear layer activation function as we are going to do a forecast and not a classification.

* Reshaping the output dimensions so that we can predict as many features as we like.

* Changing the loss function into a MSE lost.


In [3]:
from google.colab import drive
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn.modules import activation
import torch
from torch import nn
import torch.optim as optim

In [4]:
drive.mount('/content/drive')
myDrive = '/content/drive/Shareddrives/Deep_Learning_2024/Final/Data/Mixed_ETF/'

ValueError: mount failed

In [ ]:
file_path = os.path.join(myDrive,'ETF_Data.csv')
data = pd.read_csv(file_path, sep=",", header=0)

Split Data

In [ ]:
data.head()

In [ ]:
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vaw'], label='vaw')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vcr'], label='vcr')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vht'], label='vht')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vgt'], label='vgt')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vfh'], label='vfh')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vde'], label='vde')
plt.plot(pd.to_datetime(data['Date_vaw']), data['Close_vnq'], label='vnq')
plt.xlabel('Date')
plt.ylabel('Close Value')
plt.title('Close values graph')
plt.legend()
plt.show()

In [ ]:
data.rename(columns={'Date_vaw': 'Date'}, inplace=True)
data.index = data['Date']
data.pop('Date')
Train = data[:2560]
Test = data[2560:]
print(Train.shape)
print(Test.shape)
print(len(Test))

In [ ]:
mode_list = ['multi', 'single']
ETF = ['vaw', 'vcr', 'vgt', 'vfh', 'vde', ' vht', 'vnq']
mode = 'single'
ETF_active = ETF[0];

In [ ]:
import numpy as np

# Convert DataFrame to numpy array
if (mode == mode_list[0]):
  Train_array = Train.to_numpy()
  Test_array = Test.to_numpy()
else:
  Train_array = Train[['Open_'  + ETF_active,'High_' + ETF_active,'Low_'  + ETF_active,'Close_'  + ETF_active,'Volume_'  + ETF_active]].to_numpy()
  Test_array = Test[['Open_'  + ETF_active,'High_' + ETF_active,'Low_'  + ETF_active,'Close_'  + ETF_active,'Volume_'  + ETF_active]].to_numpy()

if (mode == mode_list[0]):
  Y_train = Train[['Close_vaw','Close_vcr','Close_vgt','Close_vfh','Close_vde','Close_vht','Close_vnq']].to_numpy()
  Y_test = Test[['Close_vaw','Close_vcr','Close_vgt','Close_vfh','Close_vde','Close_vht','Close_vnq']].to_numpy()
else:
  Y_train = Train[['Close_'  + ETF_active]].to_numpy()
  Y_test = Test[['Close_'  + ETF_active]].to_numpy()

mean = np.mean(Train_array, axis=(0, 1))  # Mean for each feature across all samples and timesteps
std = np.std(Train_array, axis=(0, 1))  # Std for each feature across all samples and timesteps
# Prevent division by zero by replacing zero std with 1
std = np.where(std == 0, 1, std)

# Normalize training data
Train_normalized = (Train_array - mean) / std

# Normalize test data
Test_normalized = (Test_array - mean) / std

mean_targets = np.mean(Y_train, axis=0)
std_targets = np.std(Y_train, axis=0)
std_targets[std_targets == 0] = 1  # Prevent division by zero

Y_train_normalized = (Y_train - mean_targets) / std_targets
Y_test_normalized = (Y_test - mean_targets) / std_targets

# Convert to PyTorch tensors
X_train_pt = torch.tensor(Train_normalized, dtype=torch.float32).cuda()
Y_train_pt = torch.tensor(Y_train_normalized, dtype=torch.float32).cuda()
X_test_pt = torch.tensor(Test_normalized, dtype=torch.float32).cuda()
Y_test_pt = torch.tensor(Y_test_normalized, dtype=torch.float32).cuda()

print("Shapes of normalized training and test sets:")
print("X_train_pt:", X_train_pt.shape)
print("Y_train_pt:", Y_train_pt.shape)
print("X_test_pt:", X_test_pt.shape)
print("Y_test_pt:", Y_test_pt.shape)

In [ ]:
fragment_size = 10
step = 1

if mode == mode_list[0]:
    Train_Fragmented = [Train_normalized[i:i + fragment_size] for i in range(0, len(Train_normalized) - fragment_size, step)]
    Train_Labels = [Train_normalized[i][['Close_vaw', 'Close_vcr', 'Close_vgt', 'Close_vfh', 'Close_vde', 'Close_vht', 'Close_vnq']] for i in range(fragment_size, len(Train_normalized), step)]

    Test_Fragmented = [Test_normalized[i:i + fragment_size] for i in range(0, len(Test_normalized) - fragment_size, step)]
    Test_Labels = [Test_normalized[i][['Close_vaw', 'Close_vcr', 'Close_vgt', 'Close_vfh', 'Close_vde', 'Close_vht', 'Close_vnq']] for i in range(fragment_size, len(Test_normalized), step)]
else:
    # Assuming ETF_active is a string
    feature_columns = ['Open_' + ETF_active, 'High_' + ETF_active, 'Low_' + ETF_active, 'Close_' + ETF_active, 'Volume_' + ETF_active]
    Train_Fragmented = [Train_normalized[i:i + fragment_size][:, [Train.columns.get_loc(col) for col in feature_columns]] for i in range(0, len(Train_normalized) - fragment_size, step)]
    Train_Labels = [Train_normalized[i, Train.columns.get_loc('Close_' + ETF_active)] for i in range(fragment_size, len(Train_normalized), step)]

    Test_Fragmented = [Test_normalized[i:i + fragment_size][:, [Test.columns.get_loc(col) for col in feature_columns]] for i in range(0, len(Test_normalized) - fragment_size, step)]
    Test_Labels = [Test_normalized[i, Test.columns.get_loc('Close_' + ETF_active)] for i in range(fragment_size, len(Test_normalized), step)]


In [ ]:
'''fragment_size = 10;
step = 1

if (mode == mode_list[0]):
  Train_Fragmented = [Train_normalized.iloc[i:i + fragment_size] for i in range(0, len(Train_normalized) - fragment_size, step)]
  Train_Labels = [Train_normalized.iloc[i][['Close_vaw','Close_vcr','Close_vgt','Close_vfh','Close_vde','Close_vht','Close_vnq']] for i in range(fragment_size, len(Train_normalized), step)]

  Test_Fragmented = [Test_normalized .iloc[i:i + fragment_size] for i in range(0, len(Test_normalized ) - fragment_size, step)]
  Test_Labels = [Test_normalized .iloc[i][['Close_vaw','Close_vcr','Close_vgt','Close_vfh','Close_vde','Close_vht','Close_vnq']] for i in range(fragment_size, len(Test_normalized ), step)]
else:
  Train_Fragmented = [Train_normalized.iloc[i:i + fragment_size][['Open_'  + ETF_active,'High_' + ETF_active,'Low_'  + ETF_active,'Close_'  + ETF_active,'Volume_'  + ETF_active]] for i in range(0, len(Train_normalized) - fragment_size, step)]
  Train_Labels = [Train_normalized.iloc[i][['Close_'  + ETF_active]] for i in range(fragment_size, len(Train_normalized), step)]

  Test_Fragmented = [Test.iloc[i:i + fragment_size][['Open_'  + ETF_active,'High_' + ETF_active,'Low_'  + ETF_active,'Close_'  + ETF_active,'Volume_'  + ETF_active]] for i in range(0, len(Test_normalized ) - fragment_size, step)]
  Test_Labels = [Test.iloc[i][['Close_'  + ETF_active]] for i in range(fragment_size, len(Test_normalized ), step)]'''


In [ ]:
# Print dimensions for verification
print(f'Number of Train Fragments: {len(Train_Fragmented)}')
print(f'Number of Train Labels: {len(Train_Labels)}')
print(f'Number of Test Fragments: {len(Test_Fragmented)}')
print(f'Number of Test Labels: {len(Test_Labels)}')


In [ ]:
#Print start and end of training and test data for verification
print(f'\nTraining data\n')
print(Train_Fragmented[1].columns.get_loc('Close_' + ETF_active))
print(Train_Labels[1])
print(Train_Fragmented[2].columns.get_loc('Close_' + ETF_active))
print(Train_Labels[2])

print(Train_Fragmented[2548].columns.get_loc('Close_' + ETF_active))
print(Train_Labels[2548])
print(Train_Fragmented[2549].columns.get_loc('Close_' + ETF_active))
print(Train_Labels[2549])

print(f'\nTest data\n')
print(Test_Fragmented[0].columns.get_loc('Close_' + ETF_active))
print(Test_Labels[0])
print(Test_Fragmented[1].columns.get_loc('Close_' + ETF_active))
print(Test_Labels[1])

print(Test_Fragmented[629].columns.get_loc('Close_' + ETF_active))
print(Test_Labels[629])
print(Test_Fragmented[630].columns.get_loc('Close_' + ETF_active))
print(Test_Labels[630])

In [ ]:
'''#Print start and end of training and test data for verification
print(f'\nTraining data\n')
print(Train_Fragmented[1]['Close_' + ETF_active])
print(Train_Labels[1])
print(Train_Fragmented[2]['Close_' + ETF_active])
print(Train_Labels[2])

print(Train_Fragmented[2548]['Close_' + ETF_active])
print(Train_Labels[2548])
print(Train_Fragmented[2549]['Close_' + ETF_active])
print(Train_Labels[2549])

print(f'\nTest data\n')
print(Test_Fragmented[0]['Close_' + ETF_active])
print(Test_Labels[0])
print(Test_Fragmented[1]['Close_' + ETF_active])
print(Test_Labels[1])

print(Test_Fragmented[629]['Close_' + ETF_active])
print(Test_Labels[629])
print(Test_Fragmented[630]['Close_' + ETF_active])
print(Test_Labels[630])'''

In [ ]:
print(Test_Fragmented[5]['Close_' + ETF_active])
print(Test_Fragmented[6]['Close_' + ETF_active])
# Plot the time series data using Matplotlib
plt.figure(figsize=(10, 6))
plt.plot(Test_Fragmented[5].index, Test_Fragmented[5]['Close_' + ETF_active], label='Close_'  + ETF_active + '_frag5')
plt.plot(Test_Fragmented[6].index, Test_Fragmented[6]['Close_' + ETF_active], label='Close_'  + ETF_active + '_frag6')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Plot')
plt.legend()
plt.show()

In [ ]:
# Define module encapsulating a Sequence Classifier using RNN or LSTMs and setting different architecture hyper-parameters
class Simple_LSTM(nn.Module):
  def __init__(self,
               input_size : int = 5,
               hidden_size : int = 64,
               output_size : int = 1,
               num_layers : int =  5,
               activation_function = nn.Softmax(dim = 1)):  #softmax activation function in order to get multiclass probability
    # Define RNN or LSTM architecture
    super().__init__()
    self.rnn =  nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                          num_layers=num_layers, batch_first = True)

    self.activation_function = activation_function

    self.last_linear = nn.Linear(hidden_size,output_size)

  def forward(self, X):
    _,last_states = self.rnn(X)
    # Get last hidden state for last layer. Ignore cell state in case of LSTMs
    last_hidden_state = last_states[0][-1, :, :]
    # Get sequence label probability using the last hidden state
    next_prediction = self.last_linear(last_hidden_state)
    return next_prediction


In [ ]:
def train_simple_LSTM(X_train, Y_train, model, optimizer, loss_func, epochs=100):
    loss_its = []
    for iter in range(epochs):
      optimizer.zero_grad()
      output = model(X_train)
      loss = loss_func(output,Y_train)
      loss_its.append(loss.item())
      loss.backward()
      optimizer.step()
    print(f'Final loss: {loss.item()}')
    return np.asarray(loss_its)

In [ ]:
tensor_list = [torch.tensor(fragment.values, dtype=torch.float32) for fragment in Train_Fragmented]
X_train = torch.stack(tensor_list)
label_tensor_list = [torch.tensor(fragment.values, dtype=torch.float32) for fragment in Train_Labels]
Y_train =  torch.stack(label_tensor_list)

tensor_list_test = [torch.tensor(fragment.values, dtype=torch.float32) for fragment in Test_Fragmented]
X_test = torch.stack(tensor_list_test)
label_tensor_list_test = [torch.tensor(fragment.values, dtype=torch.float32) for fragment in Test_Labels]
Y_test =  torch.stack(label_tensor_list_test)

In [ ]:
X_train_pt = X_train.float().cuda()
Y_train_pt = Y_train.float().cuda()
X_test_pt = X_test.float().cuda()
Y_test_pt = Y_test.float().cuda()

print(X_train.shape)
print(Y_train.shape)

In [ ]:
'''if (mode == mode_list[0]):
  input_size = 35
  hidden_size = 256
  num_layers = 2
  output_size = 7
  num_epochs = 4000
else:
  input_size = 5
  hidden_size = 256
  num_layers = 2
  output_size = 1
  num_epochs = 4000

# Define model, loss function, and optimizer
model = Simple_LSTM(input_size=input_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers).cuda()
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_history = train_simple_LSTM(X_train_pt, Y_train_pt, model, optimizer, loss_func, epochs=num_epochs)'''

In [ ]:
if (mode == mode_list[0]):
  input_size = 35
  hidden_size = 256
  num_layers = 2
  output_size = 7
  num_epochs = 400
else:
  input_size = 5
  hidden_size = 64
  num_layers = 2
  output_size = 1
  num_epochs = 400

# Define model, loss function, and optimizer
model = Simple_LSTM(input_size=input_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers).cuda()
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
loss_its = []
for iter in range(num_epochs):
  optimizer.zero_grad()
  output = model(X_train_pt)
  loss = loss_func(output,Y_train_pt)
  loss_its.append(loss.item())
  loss.backward()
  optimizer.step()

  if( iter%50 == 0):
    print(f'Final loss: {loss.item()}')

#loss_history = train_simple_LSTM(X_train_pt, Y_train_pt, model, optimizer, loss_func, epochs=num_epochs)


# Histogram

In [ ]:
Y_predicted = model(X_test_pt).cpu().detach().numpy()
Y = Y_test_normalized.to_numpy()
print(Y_predicted)

In [ ]:
file_path = os.path.join(myDrive,'Y_predicted.csv')
pd.DataFrame(Y_predicted).to_csv(file_path, index=False)

In [ ]:
import matplotlib.pyplot as plt

# Example arrays for Y_predicted and Y
# Ensure these are defined as per your actual data
# Y_predicted = np.random.randn(640, 6)  # Example: If Y_predicted is a 2D array
# Y = np.random.randn(640, 6, 1)  # Example: If Y is a 3D array

# Ensure `Y` is 2D by squeezing the last dimension
Y_squeezed = Y.squeeze()

# Check the shapes (for debugging purposes)
print("Y_predicted shape:", Y_predicted.shape)
print("Y shape after squeezing:", Y_squeezed.shape)

# Plot the actual signals
plt.figure(figsize=(12, 8))
if Y_squeezed.ndim == 2:
  for i in range(Y_squeezed.shape[1]):
      plt.plot(Y_squeezed[:, i], label=f"Actual Signal {i+1}")
else:
  plt.plot(Y_squeezed[:], label=f"Actual Signal {1}")

# Ensure Y_predicted is compatible for plotting
if Y_predicted.shape[1] == 1:
    # If Y_predicted is 1D, we plot it directly
    plt.plot(Y_predicted, label="Predicted Signal", linestyle='--')
elif Y_predicted.ndim == 2 and Y_predicted.shape[1] == Y_squeezed.shape[1]:
    # If Y_predicted is 2D, plot each predicted signal
    for i in range(Y_predicted.shape[1]):
        plt.plot(Y_predicted[:, i], linestyle='--', label=f"Predicted Signal {i+1}")

plt.title('Overlayed 6 Signals with 256 Samples Each')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(Y_predicted)
plt.plot(Y)
plt.title('Overlayed 6 Signals with 256 Samples Each')
plt.xlabel('Sample Index')
plt.ylabel('Signal Value')
plt.legend()
plt.grid(True)
plt.show()